In [1]:
import pandas as pd
from apis.cfb_api import get_cfb_data, get_teams_logos, get_player_game_stats
from database.database import get_db_engine


engine = get_db_engine()
incremental = True

from datetime import datetime
current_year = datetime.now().year
year_range = range(2001, current_year+1, 1)

if not incremental:
    teams, logos = get_teams_logos()
    teams.to_sql('teams', engine, schema='raw', if_exists='replace', index=False)
    logos.to_sql('logos', engine, schema='raw', if_exists='replace', index=False)

    conferences = get_cfb_data('conferences')
    conferences.to_sql('conferences', engine, schema='raw', if_exists='replace', index=False)

    for year in year_range:
        print(f'Getting game data for {year}')
        games = get_cfb_data('games', params={'year':year})
        print(f'Writing {games.shape[0]} rows to database')
        games.to_sql('games', engine, schema='raw', if_exists='append', chunksize=5000, index=False)
        print(f'Game data for {year} written to database')

    #Get Games
    sql = 'SELECT DISTINCT season FROM raw.games WHERE completed = true ORDER BY season ASC'
    seasons = pd.read_sql(sql, engine)['season'].tolist()

    game_year_range = [y for y in year_range if y not in seasons]

    for year in game_year_range:
        sql = f'SELECT MAX(week) FROM raw.games WHERE completed = true AND season = {year}'
        max_week = pd.read_sql(sql, engine)['max'].tolist()[0]
        if max_week is None:
            max_week = 0
        sql = f'SELECT DISTINCT week FROM raw.calendar WHERE season = {year} AND week > {max_week} ORDER BY week ASC'
        weeks = pd.read_sql(sql, engine)['week'].tolist()

        for week in weeks:
            print(f'Dropping old game data for {year} week {week}')
            sql = f'DELETE FROM raw.games WHERE season = {year} AND week = {week}'
            engine.execute(sql)
            print(f'Getting game data for {year} week {week}')
            games = get_cfb_data('games', params={'year':year, 'week':week})
            print(f'Writing {games.shape[0]} rows to database')
            games.to_sql('games', engine, schema='raw', if_exists='append', chunksize=5000, index=False)
            print(f'Game data for {year} week {week} written to database')

#Get calendar
if incremental:
    sql = 'SELECT DISTINCT season FROM raw.calendar ORDER BY season DESC'
    calendar_seasons = pd.read_sql(sql, engine)['season'].tolist()
    calendar_range = [y for y in year_range if y not in calendar_seasons]
else:
    calendar_range = year_range

for year in calendar_range:
    print(f'Getting calendar data for {year}')
    calendar = get_cfb_data('calendar', params={'year':year})
    calendar['season'] = calendar['season'].astype(int)
    calendar['firstGameStart'] = pd.to_datetime(calendar['firstGameStart'].apply(lambda x: x.split('T',1)[0])).dt.date
    calendar['lastGameStart'] = pd.to_datetime(calendar['lastGameStart'].apply(lambda x: x.split('T',1)[0])).dt.date
    print(f'Writing {calendar.shape[0]} rows to database')
    calendar.to_sql('calendar', engine, schema='raw', if_exists='append', chunksize=5000, index=False)
    print(f'Calendar data for {year} written to database')

#Get player game stats
sql = 'SELECT DISTINCT season, week FROM raw.games WHERE completed = true ORDER BY season DESC, week DESC'
games = pd.read_sql(sql, engine)
sql = 'SELECT DISTINCT season, week FROM raw.game_player_stats ORDER BY season DESC, week DESC'
pulled_stats = pd.read_sql(sql, engine)

seasons = games['season'].tolist()
weeks = games['week'].tolist()

seasons = [y for y in seasons if y not in pulled_stats['season'].tolist()]
weeks = [w for w in weeks if w not in pulled_stats['week'].tolist()]

for season, week in zip(seasons, weeks):
    print(f'Getting player game stats for season {season}, week {week}')
    player_game_stats = get_player_game_stats(season, week)
    player_game_stats.to_sql('game_player_stats', engine, schema='raw', if_exists='append', index=False)
    print(f'Player game stats for season {season}, week {week} written to database')




